# Prep

In [13]:
import pandas as pd
import numpy as np
import pprint
import re

## Functions

### Extract

In [2]:
def get_raw_workouts(fp):
  all_workouts = []
  counter = 0

  with open(fp, encoding='utf-8') as f:
    clean_strings = [line.strip() for line in f.readlines()]
    no_empty_lines = list(filter(None, clean_strings))
    no_empty_lines.pop(0) # tirar a primeira linha, que é vazia

    single_workout = []
    for line in no_empty_lines:
      if 'Week' in line:
        if counter > 0:
          all_workouts.append(single_workout)
          single_workout = []
        counter += 1
      single_workout.append(line)

  return all_workouts

### Transform

In [215]:
def get_workout_dict(raw_workout_list):
    workout_listdict = []

    workout_counter = 0
    for workout in raw_workout_list:

        workout_description = workout[0].split(';')
        description_info = [item.strip() for item in workout_description[0].strip('"').split('·')]


        workout_dict = {
            'workout_counter': workout_counter, # use to find errors
            #'raw_description': f'{workout[0]}', #só pra conferir por enquanto, retirar depois
            'description': {
                'day_name': description_info[0],
                'day_number': description_info[1],
                'week_number': description_info[2],
                'plan_name': description_info[3],
                'plan_function': description_info[4],
                'start_time': workout_description[1].strip('"'),
                'duration': workout_description[2].strip('"')
            },
            'exercises': [],
            'clean_exercises': []
        }


        raw_current_exercise = {
            'exercise': '',
            'data': []
        }
        raw_exercises = []
        for line in workout[1:]:
            if len(line) > 13:
                if len(raw_current_exercise['exercise']) > 0:
                    raw_exercises.append(raw_current_exercise)
                raw_current_exercise = {'exercise': line, 'data': []}
            else:
                raw_current_exercise['data'].append(line)
        raw_exercises.append(raw_current_exercise) # necessary to append the last dict


        clean_exercises = []
        for e in raw_exercises:
            exercise_info = [item.strip() for item in e['exercise'].strip('"').split('·')]

            print(f"workout {workout_counter}, {exercise_info}")

            clean_exercise = {
                'info': exercise_info,
                'exercise_name': exercise_info[0],
                'equipment': exercise_info[1]
                #'reps_goal': exercise_info[2]
            }

            clean_exercises.append(clean_exercise)
        

        workout_dict['exercises'] = raw_exercises
        workout_dict['clean_exercises'] = clean_exercises
        workout_counter += 1

        workout_listdict.append(workout_dict)
    
    return workout_listdict

# Extract

In [190]:
raw_workouts = get_raw_workouts(r'G:\My Drive\Projetos\workouts\2024_08_25 Workouts.csv')
raw_workouts.reverse()

In [214]:
if ';' in raw_workouts[28][26]:
    print("Yes")

Yes


# Transform

In [216]:
workouts_dict = get_workout_dict(raw_workouts)
pprint.pprint(workouts_dict[2], sort_dicts=False)

workout 0, ['1. Lat Pulldowns with Wide Overhand Grip', 'Cable', '12 reps']
workout 0, ['2. Chest-Supported Low Rows with Neutral Grip', 'Machine', '12 reps']
workout 0, ['3. Triceps Pushdowns', 'Cable', '12 reps']
workout 0, ['4. Curls', 'EZ bar', '12 reps']
workout 0, ['5. Bent-Over Rows', 'Barbell', '12 reps']
workout 0, ['6. Upright Rows', 'Barbell', '12 reps']
workout 0, ['7. Lat Pulldowns with Close Neutral Grip', 'Cable', '10 reps']
workout 1, ['1. Bench Press', 'Barbell', '12 reps']
workout 1, ['2. Incline Bench Press', 'Barbell', '12 reps']
workout 1, ['3. Standing Shoulder Press', 'Barbell', '12 reps']
workout 1, ['4. Butterfly with Wide Grip', 'Machine', '10 reps']
workout 1, ['5. Lateral Raises', 'Dumbbells', '12 reps']
workout 1, ['6. Front Raises', 'Barbell', '12 reps']
workout 2, ['1. Leg Press', 'Machine', '12 reps']
workout 2, ['2. Standing Calf Raises', 'Machine', '12 reps']
workout 2, ['3. Hip Adduction', 'Machine', '12 reps']
workout 2, ['4. Hip Abduction', 'Machine

IndexError: list index out of range

In [171]:
pprint.pprint(workouts_dict[32]['clean_exercises'][0]['info'], sort_dicts=False)

['1. Bench Press', 'Barbell', '12 reps']
